In [1]:
# Requests for handling HTTP get and other requests
import requests
import time # import for playing nice and not getting blocked
import pandas as pd
# from BeautifulSoup4 import BeatifulSoup if installed through pip install BeautifulSoup4
# 
from bs4 import BeautifulSoup

In [ ]:
# if bs4 not found install it with
#  !pip install beatifulsoup4
# https://www.crummy.com/software/BeautifulSoup/

In [2]:
centrs = "https://www.ss.com/lv/real-estate/flats/riga/centre/sell/"

In [3]:
req = requests.get(centrs)
req.status_code

200

In [4]:
req.text[:200]

'<!DOCTYPE html>\r\n<HTML><HEAD>\r\n<title>SS.COM Dzīvokļi - Rīga - Centrs, Cenas, Pārdod - Sludinājumi</title>\r\n<meta http-equiv="Content-Type" CONTENT="text/html; charset=UTF-8">\r\n<meta name="viewport" c'

In [5]:
soup = BeautifulSoup(req.text, 'lxml') # lxml is improved parser, a little bit more lenient
soup.title

<title>SS.COM Dzīvokļi - Rīga - Centrs, Cenas, Pārdod - Sludinājumi</title>

In [6]:
type(soup)

bs4.BeautifulSoup

In [7]:
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all
tables = soup.find_all("table")
len(tables) 

7

In [8]:

headline = soup.find("tr", {"id":"head_line"}) # this is a shorter way of finding by tr AND this element having particular id
headline

<tr id="head_line">
<td class="msg_column" colspan="3" width="70%">
<span style="float:left;"> Sludinājumi
</span>
<span align="right" class="msg_column" style="float:right;text-align:right;padding-right:3px;">
<noindex>
<a class="a19" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4S.html" rel="nofollow">datums</a></noindex></span>
</td>
<td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4SFDwT.html" rel="nofollow" title="">Iela</a></noindex></td><td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4SelM=.html" rel="nofollow" title="">Ist.</a></noindex></td><td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4QelM=.html" rel="nofollow" title="">m2</a></noindex></td><td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4XelM=.html" rel="nofollow" title

In [9]:
type(headline)

bs4.element.Tag

In [10]:
headtds = headline.find_all("td")
# https://developer.mozilla.org/en-US/docs/Web/HTML/Element/td
headtds

[<td class="msg_column" colspan="3" width="70%">
 <span style="float:left;"> Sludinājumi
 </span>
 <span align="right" class="msg_column" style="float:right;text-align:right;padding-right:3px;">
 <noindex>
 <a class="a19" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4S.html" rel="nofollow">datums</a></noindex></span>
 </td>,
 <td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4SFDwT.html" rel="nofollow" title="">Iela</a></noindex></td>,
 <td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4SelM=.html" rel="nofollow" title="">Ist.</a></noindex></td>,
 <td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4QelM=.html" rel="nofollow" title="">m2</a></noindex></td>,
 <td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4XelM=.html" rel="nofollow" title="

In [11]:
headcolumns = [el.text for el in headtds[1:]] #.text gets us content even from children and grandchildren
headcolumns

['Iela', 'Ist.', 'm2', 'Stāvs', 'Sērija', 'Cena, m2', 'Cena']

In [12]:
# lets combine the above cells into a function which will always get us columns
def getColList(soup):
    column_list = ["description","url"] # we decided to that we need these two column names no matter the html
    headline = soup.find("tr", {"id":"head_line"})
    headtds = headline.find_all("td")
    headcolumns = [el.text for el in headtds[1:]] # this will get all column names starting with 2nd in HTML
    column_list += headcolumns
    return column_list

In [13]:
column_names = getColList(soup)
column_names

['description',
 'url',
 'Iela',
 'Ist.',
 'm2',
 'Stāvs',
 'Sērija',
 'Cena, m2',
 'Cena']

In [14]:
trows = soup.find_all('tr')
len(trows)


39

In [15]:
# hardest part in this 
# how to filter only specific rows
# we need to find something that is unique to our apartment rows but not to the extra rows
# remember we id is not guaranteed
apt_rows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]
len(apt_rows)

30

In [16]:
apt_rows[-1]

<tr id="tr_48605282"><td class="msga2 pp0"><input id="c48605282" name="mid[]" type="checkbox" value="48605282_1106_0"/></td><td class="msga2"><a href="/msg/lv/real-estate/flats/riga/centre/ghepd.html" id="im48605282"><img alt="" class="isfoto foto_list" src="https://i.ss.com/gallery/4/714/178373/35674410.th2.jpg"/></a></td><td class="msg2"><div class="d1"><a class="am" data="eHVlJTk5JTg5JUNFJTdFJTdDJUE0JUExeXphJTlBJThCJThCJTdGeiVBOSVBNHM=|BB1aURMGtp" href="/msg/lv/real-estate/flats/riga/centre/ghepd.html" id="dm_48605282">Предлагается к продаже просторная, светлая квартира в посольском</a></div></td><td c="1" class="msga2-o pp6" nowrap="">Rūpniecības 7</td><td c="1" class="msga2-o pp6" nowrap="">4</td><td c="1" class="msga2-o pp6" nowrap="">110</td><td c="1" class="msga2-o pp6" nowrap="">4/5</td><td c="1" class="msga2-o pp6" nowrap="">Jaun.</td><td c="1" class="msga2-o pp6" nowrap="">2,909 €</td><td c="1" class="msga2-o pp6" nowrap="">320,000  €</td></tr>

In [17]:
# lets make a function from the above doodle and make it work on most pages on SS
def getRowList(soup):
    trows = soup.find_all('tr')
    aprows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]
    return aprows

In [18]:
row_tds = apt_rows[0].find_all('td')
len(row_tds)

10

In [20]:
row_tds

[<td class="msga2 pp0"><input id="c48092224" name="mid[]" type="checkbox" value="48092224_1106_0"/></td>,
 <td class="msga2"><a href="/msg/lv/real-estate/flats/riga/centre/dnbbh.html" id="im48092224"><img alt="" class="isfoto foto_list" src="https://i.ss.com/gallery/4/690/172288/34457411.th2.jpg"/></a></td>,
 <td class="msg2"><div class="d1"><a class="am" data="ciU5QWhmJTdDJUJGJTdFaHN5JTk3Z2t0eSU4Mmt2cSU5Nmo=|Af55CCM3A" href="/msg/lv/real-estate/flats/riga/centre/dnbbh.html" id="dm_48092224">Pārplanots un izremontēts 3.istabu dzīvoklis ķieģeļu stīprā iekš</a></div></td>,
 <td c="1" class="msga2-o pp6" nowrap="">Pērnavas 39</td>,
 <td c="1" class="msga2-o pp6" nowrap="">3</td>,
 <td c="1" class="msga2-o pp6" nowrap="">64</td>,
 <td c="1" class="msga2-o pp6" nowrap="">2/6</td>,
 <td c="1" class="msga2-o pp6" nowrap="">P. kara</td>,
 <td c="1" class="msga2-o pp6" nowrap="">1,406 €</td>,
 <td c="1" class="msga2-o pp6" nowrap="">90,000  €</td>]

In [21]:
row_tds[0].text

''

In [19]:
row_tds[1].text

''

In [22]:
row_tds[1].attrs

{'class': ['msga2']}

In [23]:
img = row_tds[1].find("img")
img

<img alt="" class="isfoto foto_list" src="https://i.ss.com/gallery/4/690/172288/34457411.th2.jpg"/>

In [24]:
img.get("src")

'https://i.ss.com/gallery/4/690/172288/34457411.th2.jpg'

In [25]:
row_tds[2].text

'Pārplanots un izremontēts 3.istabu dzīvoklis ķieģeļu stīprā iekš'

In [26]:
row_tds[-1].text

'90,000  €'

In [27]:
a = row_tds[1].find('a')
a

<a href="/msg/lv/real-estate/flats/riga/centre/dnbbh.html" id="im48092224"><img alt="" class="isfoto foto_list" src="https://i.ss.com/gallery/4/690/172288/34457411.th2.jpg"/></a>

In [28]:
a.attrs

{'href': '/msg/lv/real-estate/flats/riga/centre/dnbbh.html',
 'id': 'im48092224'}

In [29]:
a.get('href')

'/msg/lv/real-estate/flats/riga/centre/dnbbh.html'

In [30]:
def getRow(row,colist=column_names):
    row_tds = row.find_all('td')
    rowDict = {}
    if len(row_tds) <3: # a little sanity check
        print("Hmm bad row")
        return rowDict
    
    rowDict[colist[0]] = row_tds[2].text # so the big assumption is that we always get description in 3rd column
    rowDict[colist[1]] = "https://ss.com" + row_tds[1].find('a').get('href')
    for td,key in zip(row_tds[3:],colist[2:]): 
        rowDict[key] = td.text
    return rowDict

In [31]:
getRow(apt_rows[0])

{'description': 'Pārplanots un izremontēts 3.istabu dzīvoklis ķieģeļu stīprā iekš',
 'url': 'https://ss.com/msg/lv/real-estate/flats/riga/centre/dnbbh.html',
 'Iela': 'Pērnavas 39',
 'Ist.': '3',
 'm2': '64',
 'Stāvs': '2/6',
 'Sērija': 'P. kara',
 'Cena, m2': '1,406 €',
 'Cena': '90,000  €'}

In [32]:
def getRows(rowlist,colist=column_names):
    return [getRow(row) for row in rowlist]


In [33]:
row_ads = getRows(apt_rows)
row_ads[-3:]

[{'description': 'Сниженная цена. Тихий дом, памятник архитектуры, в ближайшее вре',
  'url': 'https://ss.com/msg/lv/real-estate/flats/riga/centre/gkbpx.html',
  'Iela': 'Antonijas 15',
  'Ist.': '2',
  'm2': '39',
  'Stāvs': '5/5',
  'Sērija': 'P. kara',
  'Cena, m2': '2,307 €',
  'Cena': '89,990  €'},
 {'description': 'Pārdod modernu 3 istabu dzīvokli jaunā dzīvojamo māju kvartālā P',
  'url': 'https://ss.com/msg/lv/real-estate/flats/riga/centre/bexxic.html',
  'Iela': 'Pulkv. Brieža 35',
  'Ist.': '3',
  'm2': '73',
  'Stāvs': '4/7',
  'Sērija': 'Jaun.',
  'Cena, m2': '2,425 €',
  'Cena': '177,000  €'},
 {'description': 'Предлагается к продаже просторная, светлая квартира в посольском',
  'url': 'https://ss.com/msg/lv/real-estate/flats/riga/centre/ghepd.html',
  'Iela': 'Rūpniecības 7',
  'Ist.': '4',
  'm2': '110',
  'Stāvs': '4/5',
  'Sērija': 'Jaun.',
  'Cena, m2': '2,909 €',
  'Cena': '320,000  €'}]

In [34]:
dtemp = pd.DataFrame(row_ads, columns=column_names)
dtemp.shape

(30, 9)

In [35]:
dtemp.head()

,description,url,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena
0,Pārplanots un izremontēts 3.istabu dzīvoklis ķ...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Pērnavas 39,3,64,2/6,P. kara,"1,406 €","90,000 €"
1,"Skaists divlīmeņu Town House dzīvoklis ""Grīziņ...",https://ss.com/msg/lv/real-estate/flats/riga/c...,Asara 12,4,142,1/2,Renov.,"1,620 €","230,000 €"
2,"Skaists divlīmeņu Town House dzīvoklis ""Grīziņ...",https://ss.com/msg/lv/real-estate/flats/riga/c...,Asara 12,4,134,1/2,Renov.,"1,493 €","200,000 €"
3,Продаем квартиру в доме 1955 года на на улице ...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Puškina 19,3,110,2/6,Staļina,"1,000 €","110,000 €"
4,"Mājīgs, vienistabas dzīvoklis specprojekta pie...",https://ss.com/msg/lv/real-estate/flats/riga/c...,Indrānu 2,1,33,5/5,Specpr.,"1,270 €","41,900 €"


In [36]:
def getDFfromURL(url):
    # print("getting data from", url)
    req = requests.get(url)
    if req.status_code != 200:
        print("Request Fail with", req.status_code)
        return None # maybe return empty dataframe here
    soup = BeautifulSoup(req.text, 'lxml')
    column_names = getColList(soup)
    rowlist = getRowList(soup)
    rows = getRows(rowlist)
    return pd.DataFrame(rows, columns=column_names)
    

In [37]:
df = getDFfromURL(centrs)

In [38]:
df.shape

(30, 9)

In [39]:
df.head()

,description,url,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena
0,Virtuve iekļauta cenā \r\n\r\nTikko pēc remont...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Ģertrūdes 99,2,43,3/6,Renov.,"2,034 €","87,450 €"
1,"Liels vienistabas dzīvoklis, ko var pārbūvēt p...",https://ss.com/msg/lv/real-estate/flats/riga/c...,Tomsona 19,1,40,1/5,Hrušč.,"1,100 €","44,000 €"
2,Pārplanots un izremontēts 3.istabu dzīvoklis ķ...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Pērnavas 39,3,64,2/6,P. kara,"1,406 €","90,000 €"
3,"Skaists divlīmeņu Town House dzīvoklis ""Grīziņ...",https://ss.com/msg/lv/real-estate/flats/riga/c...,Asara 12,4,142,1/2,Renov.,"1,620 €","230,000 €"
4,"Skaists divlīmeņu Town House dzīvoklis ""Grīziņ...",https://ss.com/msg/lv/real-estate/flats/riga/c...,Asara 12,4,134,1/2,Renov.,"1,493 €","200,000 €"


In [40]:

# https://developer.mozilla.org/en-US/docs/Web/HTML/Element/a
anchors = soup.find_all("a")
len(anchors)

98

In [41]:
anchors[:5]

[<a href="/" title="Sludinājumi"><img alt="Sludinājumi" border="0" class="page_header_logo" src="https://i.ss.com/img/p.gif"/></a>,
 <a class="a_menu" href="/lv/real-estate/flats/new/" title="Iesniegt Sludinājumu">Iesniegt Sludinājumu</a>,
 <a class="a_menu" href="/lv/login/" title="Mani Sludinājumi">Mani Sludinājumi</a>,
 <a class="a_menu" href="/lv/real-estate/flats/riga/centre/search/" title="Meklēt sludinājumus">Meklēšana</a>,
 <a class="a_menu" href="/lv/favorites/" title="Memo">Memo</a>]

In [42]:
navs = [anchor for anchor in anchors if anchor.get("name") == "nav_id"] # get will get us None if no "name" exists
len(navs)

10

In [43]:
navs[0]

<a class="navi" href="/lv/real-estate/flats/riga/centre/sell/page27.html" name="nav_id" rel="prev"><img border="0" height="5" src="https://i.ss.com/img/s_left.png" style="padding-bottom:2px;" width="9"/> Iepriekšējie</a>

In [44]:
navs[0].attrs

{'name': 'nav_id',
 'rel': ['prev'],
 'class': ['navi'],
 'href': '/lv/real-estate/flats/riga/centre/sell/page27.html'}

In [45]:
navs[0]['href']

'/lv/real-estate/flats/riga/centre/sell/page27.html'

In [46]:
# could use regular expression here but we can use split 
afterpage = navs[0]['href'].split("/page")[1]
afterpage

'27.html'

In [47]:
beforedot = afterpage.split(".html")[0]
beforedot

'27'

In [48]:
lastpage = int(beforedot)
lastpage

27

In [49]:
def getAllLocalUrls(url):
    """Get a list of all urls including paginated pages"""
    results = [url] # default is just the url if no extra pages found
    req = requests.get(url)
    if req.status_code != 200:
        print(f"Bad response! {req.status_code}")
        return []
    soup = BeautifulSoup(req.text, 'lxml')
    # we just need a one element
    prevanchor = soup.find('a', {"rel":"prev"})
    if prevanchor == None:
        return results
    href = prevanchor.attrs.get('href')
    lastPageNum = int(href.split('page')[-1].split('.')[0])
    print("Last page is",lastPageNum)
    nurls = [f"{url}page{n}.html" for n in range(2,lastPageNum+1)]
    results += nurls
    return results

In [51]:
centrs

'https://www.ss.com/lv/real-estate/flats/riga/centre/sell/'

In [52]:
local_urls = getAllLocalUrls(centrs)
len(local_urls),local_urls[:3],local_urls[-1]

Last page is 27


(27,
 ['https://www.ss.com/lv/real-estate/flats/riga/centre/sell/',
  'https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page2.html',
  'https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page3.html'],
 'https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page27.html')

In [53]:
dlast = getDFfromURL(local_urls[-1])
dlast.shape

(8, 9)

In [55]:
dlast

,description,url,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena
0,Dzīvoklis atrodas pirmskara mājā ar jaunu lift...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Brīvības 142,2,48,5/6,P. kara,"2,083 €","100,000 €"
1,Dzīvoklis atrodas pirmskara mājā ar jaunu lift...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Brīvības 142,2,49,4/6,P. kara,"2,041 €","100,000 €"
2,Продается полуподвальное помещение (с окном и ...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Baznīcas 13,1,28,1/4,P. kara,"1,321 €","37,000 €"
3,"Pārdodam 3 ist. dzīvokli ar mēbelēm, pazemes a...",https://ss.com/msg/lv/real-estate/flats/riga/c...,J. Daliņa 8,3,108,12/24,Jaun.,"2,083 €","225,000 €"
4,Pārdodu labiekārtotu un mēbelētu 3 istabu dzīv...,https://ss.com/msg/lv/real-estate/flats/riga/c...,J. Daliņa 8,3,109,13/24,Jaun.,"2,064 €","225,000 €"
5,Отделка Премиум класса \r\n\r\nПолная реноваци...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Ģertrūdes 23,3,62,2/6,Jaun.,"2,677 €","166,000 €"
6,Pārdod 3 istabu dzīvokli Brīvības ielā 39a. \r...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Brīvības 39a,2,100,3/6,P. kara,"1,590 €","159,000 €"
7,Piedāvājumā 4 istabu dzīvoklis vēsturiskā namā...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Čaka 36,4,84,3/5,P. kara,"1,821 €","152,935 €"


In [56]:
def get_all_ads_df(start_url, save_excel_path=None):
    df_list=[]
    local_urls = getAllLocalUrls(start_url)
    for url in local_urls:
        print(f"Gathering data from {url}")
        df_list.append(getDFfromURL(url))
        time.sleep(0.3) # we need this to play nice!
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html
    big_df = pd.concat(df_list)
    if save_excel_path:
        big_df.to_excel(save_excel_path)
    return big_df
    

In [57]:
centrs

'https://www.ss.com/lv/real-estate/flats/riga/centre/sell/'

In [58]:
d = get_all_ads_df(centrs, "centrs.xlsx")

Last page is 27
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page2.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page3.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page4.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page5.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page6.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page7.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page8.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page9.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page10.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page11.html
Gathering data from http

In [59]:
d.shape

(788, 9)

In [60]:
d.head()

,description,url,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena
0,Virtuve iekļauta cenā \r\n\r\nTikko pēc remont...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Ģertrūdes 99,2,43,3/6,Renov.,"2,034 €","87,450 €"
1,"Liels vienistabas dzīvoklis, ko var pārbūvēt p...",https://ss.com/msg/lv/real-estate/flats/riga/c...,Tomsona 19,1,40,1/5,Hrušč.,"1,100 €","44,000 €"
2,Pārplanots un izremontēts 3.istabu dzīvoklis ķ...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Pērnavas 39,3,64,2/6,P. kara,"1,406 €","90,000 €"
3,"Skaists divlīmeņu Town House dzīvoklis ""Grīziņ...",https://ss.com/msg/lv/real-estate/flats/riga/c...,Asara 12,4,142,1/2,Renov.,"1,620 €","230,000 €"
4,"Skaists divlīmeņu Town House dzīvoklis ""Grīziņ...",https://ss.com/msg/lv/real-estate/flats/riga/c...,Asara 12,4,134,1/2,Renov.,"1,493 €","200,000 €"


In [61]:
d.tail()

,description,url,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena
3,"Pārdodam 3 ist. dzīvokli ar mēbelēm, pazemes a...",https://ss.com/msg/lv/real-estate/flats/riga/c...,J. Daliņa 8,3,108,12/24,Jaun.,"2,083 €","225,000 €"
4,Pārdodu labiekārtotu un mēbelētu 3 istabu dzīv...,https://ss.com/msg/lv/real-estate/flats/riga/c...,J. Daliņa 8,3,109,13/24,Jaun.,"2,064 €","225,000 €"
5,Отделка Премиум класса \r\n\r\nПолная реноваци...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Ģertrūdes 23,3,62,2/6,Jaun.,"2,677 €","166,000 €"
6,Pārdod 3 istabu dzīvokli Brīvības ielā 39a. \r...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Brīvības 39a,2,100,3/6,P. kara,"1,590 €","159,000 €"
7,Piedāvājumā 4 istabu dzīvoklis vēsturiskā namā...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Čaka 36,4,84,3/5,P. kara,"1,821 €","152,935 €"


# TODO 
# Try with different starting address not only centrs
## Maybe combine regions
## See how it would work with maybe cars
## Data engineering make new columns based on existing ones, clean some columns
### Changing floors 2/6 to columns 2 and 6, clean up Euro signs
## Sorting, Describing, Grouping by regions etc